In [2]:
import numpy as np
import pandas as pd
from fastai import *
from fastai.io import *
from fastai.column_data import *
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt

In [3]:
PATH='../lec6/nietzsche.txt'

In [4]:
txt=open(PATH).read()
chars=sorted(list(set(txt)))
chars.insert(0,"\0")
print (f"Total chars are {len(chars)}")
vocab_size=len(chars)

Total chars are 85


In [5]:
chars_id={ch:i for i,ch in enumerate(chars)}
id_chars={i:ch for i,ch in enumerate(chars)}

In [7]:
n_hidden=256
n_fac =42

In [10]:
bs=512

In [12]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = V(torch.zeros(1, bs, n_hidden))

In [13]:
m=CharSeqStatefulRnn(vocab_size,n_fac,bs)

In [14]:
PATH='./text_data/'
TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

from torchtext import vocab,data
from fastai.nlp import *
from fastai.lm_rnn import *

In [18]:
len_trn=int(len(txt)*.20)
trn_file=open(f'{TRN}text.txt','w')
trn_file.write(txt[:len_trn])
trn_file.close()

In [19]:
val_file=open(f'{VAL}text.txt','w')
val_file.write(txt[len_trn:])
val_file.close()

In [34]:
TEXT=data.Field(lower=True,tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES= dict(train=TRN_PATH,validation=VAL_PATH,test=VAL_PATH)
md=LanguageModelData.from_text_files(PATH,TEXT,**FILES,bs=bs,bptt=bptt,min_freq=3)

In [35]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [36]:
m=CharSeqStatefulRnn(vocab_size,n_fac,bs)
opt=torch.optim.Adam(m.parameters(),1e-3)

In [41]:
fit(m,md,2,opt,F.nll_loss)

epoch      trn_loss   val_loss                              
    0      2.081986   2.093151  
    1      1.946331   1.985067                              



[array([1.98507])]

In [94]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [95]:
m=CharSeqStatefulLSTM(md.nt,n_fac,bs,2)
lo=LayerOptimizer(optim.Adam,m,1e-2,1e-5)

In [96]:
os.makedirs(f'{PATH}models',exist_ok=True)

In [97]:
on_end=lambda sched,cycle : save_model(m,f'{PATH}models/cyc_{cycle}')
cb=[CosAnneal(lo,len(md.trn_dl),cycle_mult=2,on_cycle_end=on_end)]
fit(m,md,2,lo.opt,F.nll_loss,callbacks=cb)

epoch      trn_loss   val_loss                              
    0      2.192248   2.127551  
    1      1.968211   1.9078                                



[array([1.9078])]

In [98]:
def get_next(inp):
    idxs=TEXT.numericalize(inp,device=-1)
    ix=to_np(m(*[(idxs)]))[0].argmax()
    return TEXT.vocab.itos[ix]

In [139]:
get_next("th")

'h'